In [15]:
from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from scipy import stats
from imblearn.over_sampling import SMOTE

df_all=pd.read_csv('C:/diabetic_data_processed.csv')
print(type(df_all))
to_del = ['admission_source_id','encounter_id', 'patient_nbr','medical_specialty','payer_code','weight','index','admission_type_id','discharge_disposition_id']        
print (to_del)

#Filter_selected cols
filtered_cols = [c for c in df_all.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df_all[filtered_cols]
print ("df_2",df_2.shape)

# Filter complete null columns
cols = np.where((np.sum(df_2.isnull(), axis=0).values) == df_2.shape[0])[0]
print (cols)
filt_cols = [c for c in df_2.columns if c not in df_2.columns[cols]]
df_3 = df_2[filt_cols]
print ("df_3",df_3.shape)

#Fill na
df_4 = df_3.fillna(value=np.mean(df_3,axis=0),inplace=False,axis=0).values
print ("df_4",df_4.shape)
data=df_4

selector = VarianceThreshold(threshold=(.99 * (1 - .99)))
newdata=selector.fit_transform(data)
idxs = selector.get_support(indices=True)
print(data[:, idxs])
print("indices",idxs)
columnslist=df_3.columns.tolist()
print("lenindex",len(idxs))
for z in range(0,len(columnslist)):
    if z not in idxs:
        print(columnslist[z])
print("after",newdata.shape)
print("initial",data.shape)
print("Headers_FINAL: ", df_2.columns.values.tolist())
for z in range(0,len(idxs)-1): 
    for y in range(0,len(idxs)-1): 
        #print("column: ",data[:,z])
        if (z!=y and z<y):
            correlation=stats.pearsonr(data[:,z], data[:,y])
            if (abs(correlation[0])>0.4):
                print("Columns high correlation",df_3.columns[z],",",df_3.columns[y],correlation[0])


msk = np.random.rand(len(newdata)) < 0.8
df, df_test = newdata[msk].copy(), newdata[~msk].copy()
print("df.shape",df.shape)
print("df_test.shape",df_test.shape)

y_train=df[:,-1]
y_test=df_test[:,-1]
x_train=df[:,:-2]
x_test=df_test[:,:-2]

y_train=np.where(y_train > 0, 1, y_train)
y_test=np.where(y_test > 0, 1, y_test)
print(set(y_train))
print(set(y_test))

sm = SMOTE(n_jobs=-1, random_state=42,kind='regular')
x_train_res,y_train_res = sm.fit_sample(x_train,y_train)
x_test_res,y_test_res = sm.fit_sample(x_test,y_test)

print("Test", set(y_test))

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
['admission_source_id', 'encounter_id', 'patient_nbr', 'medical_specialty', 'payer_code', 'weight', 'index', 'admission_type_id', 'discharge_disposition_id']
df_2 (101766, 42)
[]
df_3 (101766, 42)
df_4 (101766, 42)
[[  0.   1.  10. ...,   0.   0.   0.]
 [  0.   1.  20. ...,   1.   1.  31.]
 [  1.   1.  30. ...,   0.   1.   0.]
 ..., 
 [  0.   0.  80. ...,   1.   1.   0.]
 [  0.   1.  90. ...,   1.   1.   0.]
 [  0.   0.  80. ...,   0.   0.   0.]]
indices [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 20 22 23 25 26 33 39
 40 41]
lenindex 27
nateglinide
chlorpropamide
acetohexamide
tolbutamide
acarbose
miglitol
troglitazone
tolazamide
examide
citoglipton
glyburide-metformin
glipizide-metformin
glimepiride-pioglitazone
metformin-rosiglitazone
metformin-pioglitazone
after (101766, 27)
initial (101766, 42)
Headers_FINAL:  ['race', 'gender', 'age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient'

In [6]:
#NaiveBayes
#print ("Reduced data: ",reduced_data.shape)
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Set the parameters by cross-validation
#'min_samples_leaf': [1,5,10,50,100,200,500],

#tuned_parameters = {
#    'n_estimators': [20, 50, 100],
#    'max_features': ['auto', 'sqrt', 'log2'],
#    'class_weight':['balanced']
#}

tuned_parameters = {  
} 

#tuned_parameters = {  
#    }  

#scores = ['precision', 'recall', 'f1']
#scores = [ 'f1_weighted','f1_micro','f1_macro','accuracy']
scores=['f1_weighted']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    #clf = GridSearchCV(GaussianNB(), tuned_parameters, cv=10,scoring='%s_macro' % score)
    #clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5,scoring='%s_weighted' % score,n_jobs= -1)
    #clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5,scoring='%s' % score,n_jobs= -1)
    clf = GridSearchCV(GaussianNB(), tuned_parameters, cv=10,scoring='%s' % score)
    clf.fit(x_train_res, y_train_res)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_train, y_pred = y_test, clf.predict(x_test)
    y_train2, y_pred = y_test_res, clf.predict(x_test_res)
    print(classification_report(y_train2, y_pred))
    print("Accuracy score: ", accuracy_score(y_test_res, y_pred))
    print()
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test_res, y_pred)

# Tuning hyper-parameters for f1_weighted

Best parameters set found on development set:

{}

Grid scores on development set:

0.530 (+/-0.069) for {}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

        0.0       0.55      0.86      0.67     10889
        1.0       0.68      0.28      0.40     10889

avg / total       0.61      0.57      0.53     21778

Accuracy score:  0.573376802278



In [7]:
#RandomForest
#Decision tree
#print ("Reduced data: ",reduced_data.shape)
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from scipy import interp

rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=2, oob_score = True) 

# Set the parameters by cross-validation
#'min_samples_leaf': [1,5,10,50,100,200,500],

tuned_parameters = {
    'n_estimators': [20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'class_weight':['balanced']
}

#tuned_parameters = {  
#    }  

#scores = ['precision', 'recall', 'f1']
#scores = [ 'f1_weighted','f1_micro','f1_macro','accuracy','roc_auc']
scores=['f1_weighted']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    #clf = GridSearchCV(GaussianNB(), tuned_parameters, cv=10,scoring='%s_macro' % score)
    #clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5,scoring='%s_weighted' % score,n_jobs= -1)
    clf = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv=5,scoring='%s' % score,n_jobs= -1)
    clf.fit(x_train_res, y_train_res)

    
    
    array=clf.best_estimator_.feature_importances_
    print(type(array))
    for i in range (0,len(array)):
        print(df_2.columns[i],": ",array[i])
    #print("clf: ",type(clf))
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_train, y_pred = y_test, clf.predict(x_test)
    y_train2, y_pred = y_test_res, clf.predict(x_test_res)
    print(classification_report(y_train2, y_pred))
    print("Accuracy score: ", accuracy_score(y_test_res, y_pred))
    print()
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test_res, y_pred)
    Total=0
    for i in range(0,2): 
        Total=Total+cm[i,0]+cm[i,1]
    specificity = [0,0]
    for i in range(0,2): 
        TP=cm[i,i] 
        TN=Total-cm[i,0]-cm[i,1]+cm[i,i]
        FP=cm[0,i]+cm[1,i]-cm[i,i]
        FN=cm[i,0]+cm[i,1]-cm[i,i]
        specificity[i]=(TN/(TN+TP))
        print("i: ",i, ", specificity: ",specificity[i])
    #accuracy= (TP+TN)/Total
    #print("Accuracy: ",accuracy)
    w = [0,0,0,0]
    SpecificityGlobal=0
    for i in range(0,2): 
        w[i]= (cm[i,0]+cm[i,1])/Total
        SpecificityGlobal  =  SpecificityGlobal+ w[i] * specificity[i]
    print("SpecificityGlobal: ", SpecificityGlobal)

# Tuning hyper-parameters for f1_weighted

<class 'numpy.ndarray'>
race :  0.024520190489
gender :  0.0201832101304
age :  0.0531181332559
time_in_hospital :  0.0652301110853
num_lab_procedures :  0.105107358081
num_procedures :  0.0415922323463
num_medications :  0.0910764450077
number_outpatient :  0.0234311495444
number_emergency :  0.021068213243
number_inpatient :  0.0647559244651
diag_1 :  0.0991663032532
diag_2 :  0.0982409940528
diag_3 :  0.0963315910476
number_diagnoses :  0.0469015398329
max_glu_serum :  0.00899325585506
A1Cresult :  0.0193238144907
metformin :  0.0166836785487
repaglinide :  0.00333525479596
nateglinide :  0.00810914458032
chlorpropamide :  0.0141804250599
glimepiride :  0.012836179064
acetohexamide :  0.00924986272355
glipizide :  0.00847798300376
glyburide :  0.0296675999705
tolbutamide :  0.0184194060735
Best parameters set found on development set:

{'class_weight': 'balanced', 'n_estimators': 100, 'max_features': 'log2'}

Grid scores on development set:

In [ ]:
#SVM
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf','linear'], 'gamma': [1e-1,1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}]

#tuned_parameters = [{'kernel': ['rbf'],
#                     'C': [1]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=10,
                       scoring='%s_macro' % score)
    clf.fit(x_train_res, y_train_res)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_train, y_pred = y_test, clf.predict(x_test)
    y_train2, y_pred = y_test_res, clf.predict(x_test_res)
    print(classification_report(y_train2, y_pred))
    print()

# Tuning hyper-parameters for precision

